In [16]:
!pip install selenium

  Using cached trio_websocket-0.9.2-py3-none-any.whl (16 kB)
  Using cached outcome-1.1.0-py2.py3-none-any.whl (9.7 kB)
  Using cached wsproto-1.1.0-py3-none-any.whl (24 kB)
  Using cached h11-0.13.0-py3-none-any.whl (58 kB)


In [17]:
from selenium import webdriver
import re
import numpy as np
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup as bs
import time
import math
import requests
import json

In [128]:
base_url="https://emart.ssg.com/category/list.ssg?dispCtgId="
id="6000095935"
detail="&shpp=ssgem&sort=prcasc&page="
page=1

url=base_url+id+detail+str(page)
hdr = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36'}
req=requests.get(url=url,headers=hdr)
req.encoding="utf-8"
r=req.text
b=bs(r,"html.parser")

In [145]:
base_url="https://emart.ssg.com/category/list.ssg?dispCtgId="
id="6000095935"
detail="&shpp=ssgem&sort=prcasc&page="
page=1
t=[]
c=[]
p=[]
im=[]
hdr = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36'}
while(1):
  url=base_url+id+detail+str(page)
  req=requests.get(url=url,headers=hdr)
  req.encoding="utf-8"
  r=req.text
  b=bs(r,"html.parser")
  product=b.select("#ty_thmb_view > ul > li")

  if len(product)==0:
    print("no left")
    break
  for product in product:
    title= product.select("div.cunit_info > div.cunit_md.notranslate > div > a > em.tx_ko")[0]
    cost=product.select("div.cunit_info > div.cunit_price.notranslate > div.opt_price > em")[0]
    img=product.select("div.cunit_prod > div.thmb > a > img.i1")[0]
    try:
      pop=(product.select("div.cunit_info > div.cunit_app > div > span > em")[0].text)
    except:
      pop="0"
    pop=int(pop.replace(",",""))
    img_url="https:"+img['src']
    img_url=img_url.replace('290.jpg','1200.jpg')
    t.append(title.text)
    c.append(cost.text)
    p.append(pop)
    im.append(img_url)
  page+=1

df=pd.DataFrame({'name':t,'price':c,'popularity':p,'img_url':im})

no left


In [ ]:
display(df)
# print(df.loc[0,'img_url'])

In [ ]:
display(df[df['name'].str.contains("[★/]")])

In [ ]:
display(df[df['name'].str.len()>=25])

In [132]:
count=0
page=1
hdr = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36'}
url_base="https://emart.ssg.com/disp/category.ssg?dispCtgId=6000095934&shpp=ssgem&sort=prcasc&page="
while(1):
  url=url_base+str(page)
  req=requests.get(url=url,headers=hdr)
  req.encoding="utf-8"
  r=req.text
  b=bs(r,"html.parser")
  #상품 이름, id가져오기
  title=b.select("#ty_thmb_view > ul > li > div.cunit_info > div.cunit_md.notranslate > div > a > em.tx_ko")
  id=b.select("#ty_thmb_view > ul > li > div.cunit_prod > div.thmb > a['href']")
  img=b.select("#ty_thmb_view > ul > li > div.cunit_prod > div.thmb > a > img.i1")
  if(len(title)==0):
    print(page,count,"error")
    break
  for i in range(len(id)):
    name=title[i].text
    name=name.replace('/','_')
    img_url="https:"+img[i]['src']
    img_url=img_url.replace('290.jpg','1200.jpg')
    fileurl='일반과자/'+name+".jpg"
    # urllib.request.urlretrieve(img_url,fileurl)
    # print(img_url)
    count+=1
  page+=1

5 317 error


In [52]:
p=r"C:/BigData/last_project/last/bigdata/data"
print(p)
os.listdir(p+'/'+a)
print(p)

C:/BigData/last_project/last/bigdata/data
C:/BigData/last_project/last/bigdata/data


In [76]:
#aihub데이터에 어떠한 것들이 있는지 dataframe으로 제작 + 세부 정보들 추가
import os
ori_p=r"C:\BigData\last_project\last\bigdata\data"
p=r"C:/BigData/last_project/last/bigdata/data"
fileex=r'_meta.xml'
big_cate=os.listdir(p)

"""
id,대분류, 중분류, 소분류, 세부분류, 브랜드명, 제품명, 용량, 영양정보

"""
item=[]
b_c=[]
m_c=[]
s_c=[]
n_c=[]
brand=[]
title=[]
volume=[]
nut=[]
count=0
for a in big_cate:
    name=os.listdir(p+'/'+a)
    for b in name:
        xlist=[file for file in os.listdir(p+'/'+a+'/'+b) if file.endswith(fileex)]
        xml_file=xlist[0]
        path=os.path.join(ori_p,a,b,xml_file)
        fp=open(path,"r",encoding="utf-8")
        soup=bs(fp,'html.parser')
        element=soup.find('div_cd')
        item.append(element.item_no.text)
        b_c.append(element.div_l.text)
        m_c.append(element.div_m.text)
        s_c.append(element.div_s.text)
        n_c.append(element.div_n.text)
        brand.append(element.comp_nm.text)
        title.append(element.img_prod_nm.text)
        volume.append(element.volume.text)
        nut.append(element.nutrition_info.text)
        count+=1
    print(count)
    

df=pd.DataFrame({'id':item,'대분류':b_c,'중분류':m_c,'소분류':s_c,'세부분류':n_c,'제조사':brand,'이름':title,'용량':volume,'영양정보':nut})


In [91]:
#중복 데이터 확인
pd.options.display.max_rows = 30
df2=df[df.duplicated(['이름'])]
t=[]
temp=pd.DataFrame({'id':t,'이름':t})
for name in df2['이름']:
    a=(df[df['이름']==name])
    temp=pd.concat([temp,a],ignore_index=True)

display(temp)
temp.to_excel('C:\BigData\last_project\dup.xlsx')

In [ ]:
!zip -r /content/일반과자.zip /content/일반과자

In [136]:
from google.colab import files
files.download('/content/일반과자.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>